In [ ]:
import os
import yaml

# List availavle mappings
cgmap_data_dir = os.path.join('.', 'cgmap', 'data')
mappings = sorted([name for name in os.listdir(cgmap_data_dir) if os.path.isdir(os.path.join(cgmap_data_dir, name))])
mapping_resnames = {}

for mapping_name in mappings:
    folder_path = os.path.join(cgmap_data_dir, mapping_name)
    resnames = set()
    for fname in os.listdir(folder_path):
        if fname.endswith('.yaml'):
            yaml_path = os.path.join(folder_path, fname)
            with open(yaml_path, 'r') as f:
                try:
                    data = yaml.safe_load(f)
                    molecule = data.get('molecule')
                    if molecule:
                        resnames.add(molecule)
                except Exception as e:
                    pass
    mapping_resnames[mapping_name] = sorted(resnames)

print("Available mappings: [supported resnames for these mappings]")
for mapping_name, resnames in mapping_resnames.items():
    print(f"{mapping_name}: {', '.join(resnames) if resnames else 'No resnames found'}")

In [1]:
from cgmap.mapping.mapper import Mapper

# args_dict = {
#     'mapping': 'martini3',
#     'input': '/scratch/angiod/nmr/ucbshift.single/test/rcsb.pdb/',
#     # 'inputtraj': '/protein.trr',
#     # 'traj_format': 'trr',
#     # 'trajslice': ':10',
#     'selection': 'protein',
#     'output': '/scratch/angiod/nmr/ucbshift.single/test/rcsb.CG.npz/',
#     # 'outputtraj': 'trr',
#     'isatomistic': True,
# }
args_dict = {
    'mapping': 'martini3',
    'input': 'data/a2a.pdb',
    # 'inputtraj': 'data/mytraj.trr [xtc, ...]',
    # 'trajslice': '::10', # stride of 10
    'selection': 'protein',
    'isatomistic': True, # Tell we are mapping from atomistig to CG
}

mapping = Mapper(args_dict=args_dict)

# --- Option 1 --- #
# Map only the ith file in the list of files (and optionally trajectories)

# mapping.map(index=0)
# mapping.save()
# mapping.save_atomistic()
# mapping.save_npz(to_pos_unit='Angstrom', to_force_unit='kcal/(mol*Angstrom)')

# --- Option 2 --- #
# Map all the files one after the other

for i, (m, npz_filename) in enumerate(mapping()):
    m.save()
    m.save_atomistic()
    m.save_npz(
        from_pos_unit='Angstrom',
        to_pos_unit='Angstrom',
        from_force_unit='kcal/(mol*Angstrom)',
        to_force_unit='kcal/(mol*Angstrom)'
    )